In [ ]:
%load_ext autoreload
%reload_ext autoreload
%env XLA_PYTHON_CLIENT_ALLOCATOR=platform

env: XLA_PYTHON_CLIENT_ALLOCATOR=platform


# baseline mlp

> Conventional fully connected neural network for metalearning trajectory prediction.

In [ ]:
# | default_exp baseline

In [ ]:
# | hide
import nbdev
from fastcore.test import test_eq
from nbdev.showdoc import *

In [ ]:
# | export
import jax
import jax.numpy as jnp
import equinox as eqx

In [ ]:
from jaxDiversity.mlp import MultiActMLP, deterministic_init, init_linear_weight

In [ ]:
# | export
@eqx.filter_value_and_grad()
def compute_loss(model, x, y, afuncs):
    """Compute the l2 loss of the model on the given data."""
    pred_y, _ = jax.vmap(model, in_axes=(0, None))(x, afuncs)  # batched predictions
    loss = jnp.mean((pred_y - y) ** 2)
    return loss

In [ ]:
# | test
# test compute_loss
key = jax.random.PRNGKey(0)
model_key, init_key = jax.random.split(key)
x = jnp.ones((5, 4))

model = MultiActMLP(4, 2, [18], model_key, bias=False)
model = init_linear_weight(model, deterministic_init, init_key)
y = jnp.ones((5, 2))

afuncs = [lambda x: 0, lambda x: 0]

loss, grad = compute_loss(model, x, y, afuncs)
test_eq(loss, 1)

In [ ]:
# | hide
nbdev.nbdev_export()